In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import os, datetime
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
print('Tensorflow version: {}'.format(tf.__version__))

import matplotlib.pyplot as plt
plt.style.use('seaborn')

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Flatten
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
import sqlite3
import pandas as pd
import math
import matplotlib.pyplot as plt
plt.style.use('seaborn')
from datetime import datetime
import random

import os
os.chdir('/gstore/project/hr_brca_heterogeneity/T47D_trace_Seq_v2/Nathan')
#os.listdir()

2024-01-16 11:21:17.443586: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-16 11:21:22.881461: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-16 11:21:22.886209: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-16 11:21:33.278414: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Tensorflow version: 2.12.0


In [2]:
'''Hyperparameters'''

batch_size = 32 # was 32
seq_len = 8 # was 128

#d_k = 256
#d_v = 256
n_heads = 40 # was 60  # Number of attention heads # no difference or imporvement between 5-60
n_head_size= 5 # was 46 # Embedding size for attention # no difference between 5-46
n_ff_dim = 5  # was 55    # Hidden layer size in feed forward network inside transformer # some difference between 5-55


In [3]:
#### full data
import json

# tran_9545_plsrres0.3_HALL3.json
# tran_palbo_plsrres0.3_HALL3.json
# tran_combo_plsrres0.5_HALL3.json
f = open('json_files/tran_9545_plsrres0.3_HALL3.json')
data = json.load(f)

In [4]:
exp_days = {'Day 0': 1, 'Day 1': 2, 'Day 4': 3, 'Day 8': 4, 'Day 26': 5}

df = pd.DataFrame(data['GFPBC_libB_90850'])
df.iloc[:, 0:len(df.columns)-1]

,Pseudotime,RealTime,Survival,Score.MAPK,Score.ER_induced,Score.ER_repressed,Score.Hippo_145,Score.Hippo_50,Score.TCGA_LuBrCa_mPIK3CA_up,Score.TCGA_LuBrCa_mPIK3CA_dn,...,Score.HALLMARK_ANGIOGENESIS,Score.HALLMARK_HEME_METABOLISM,Score.HALLMARK_COAGULATION,Score.HALLMARK_IL2_STAT5_SIGNALING,Score.HALLMARK_BILE_ACID_METABOLISM,Score.HALLMARK_PEROXISOME,Score.HALLMARK_ALLOGRAFT_REJECTION,Score.HALLMARK_SPERMATOGENESIS,Score.HALLMARK_KRAS_SIGNALING_UP,Score.HALLMARK_KRAS_SIGNALING_DN
0,0.0098,1,0.3518,0.0000,0.3627,0.4066,0.2999,0.2336,0.1245,0.0844,...,0.8933,1.7672,2.3216,1.6280,1.5111,1.9565,2.2742,1.6578,2.1438,0.6785
1,0.0105,2,0.3143,0.1703,0.2965,0.3819,0.4214,0.3118,0.0888,0.1282,...,2.0071,1.7374,1.9475,1.7186,2.0668,2.1993,2.1643,1.4647,1.9337,1.6546
2,0.0116,1,0.2891,0.2174,0.5001,0.5001,0.3403,0.2268,0.0770,0.0000,...,1.6901,1.2966,2.1445,1.5817,1.9198,2.1749,1.7421,2.0901,1.9001,1.4106
3,0.0129,1,0.2772,0.4634,0.5483,0.2941,0.3515,0.2339,0.1068,0.3133,...,2.1376,1.7490,2.1339,1.6222,1.5716,2.1146,2.1199,1.9977,2.0230,1.5737
4,0.0143,2,0.2668,0.4727,0.4183,0.3784,0.4032,0.3497,0.1019,0.2250,...,1.7691,1.4083,2.1572,1.6870,1.8793,2.3432,2.2296,1.8866,1.8935,1.3916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2380,0.9984,5,0.0899,0.1211,0.2532,0.4466,0.4576,0.5640,0.3350,0.1142,...,2.0554,1.5934,1.6054,1.7091,1.5929,1.5802,1.8458,1.5784,1.9996,1.5064
2381,0.9991,5,0.0928,0.1573,0.3216,0.6288,0.5804,0.6759,0.2311,0.0567,...,1.5983,1.6818,1.7428,1.6946,1.5457,1.3229,1.9067,1.4309,1.5273,1.7536
2382,0.9995,5,0.0841,0.1324,0.4018,0.7432,0.4769,0.5029,0.2264,0.0948,...,1.1106,1.4459,1.3601,1.7555,1.7549,1.4490,1.7661,1.5650,1.8006,1.8729
2383,0.9999,5,0.0745,0.1656,0.2699,0.6598,0.4079,0.5167,0.3009,0.1464,...,1.7250,1.6574,1.7760,2.1085,1.8774,1.5012,1.6859,1.4455,1.6761,1.6979


In [5]:
barcodes = [i for i in data.keys()]
new_barcodes = []
for i in barcodes:
    df = pd.DataFrame(data[i])
    #print(i, df.shape[0])
    if df.shape[0] > 200: # 50 or 60
        new_barcodes.append(i)
print(new_barcodes)
print(len(new_barcodes))

bar_dict = {new_barcodes[i]:i for i in range(len(new_barcodes))}

['GFPBC_libB_92196', 'GFPBC_libB_20182', 'GFPBC_libB_90850', 'GFPBC_libB_10678', 'GFPBC_libB_37798', 'GFPBC_libB_31581', 'GFPBC_libB_40758', 'GFPBC_libB_94397', 'GFPBC_libB_83026', 'GFPBC_libB_10737']
10


In [6]:
'''Create training and test data'''

def transformer_train_test_split_dayiter(bar, frac_val, extra_var_bool, splitval):
    
    bardf = pd.DataFrame(data[bar])
    #bardf = bardf[bardf['RealTime'] <= exp_days[d]]
    num_cell_day = len(bardf['RealTime'].tolist())
    snum = int(num_cell_day*splitval)
    bardf = bardf.iloc[:snum,:]
    
    
    bardf = bardf.iloc[:,:len(bardf.columns)-1]
    bardf['RealTime'] = bardf['RealTime'].astype('category')
    bardf['Barcode'] = [bar_dict[bar] for _ in range(bardf.shape[0])]
    bardf['Barcode'] = bardf['Barcode'].astype('category')
    
    # normalize after train/test split
    scaler = MinMaxScaler(feature_range=(-1, 1)) 
    bardf2 = pd.DataFrame(scaler.fit_transform(bardf.iloc[:,2:len(bardf.columns)-1])) # survival and expression
    bardf2.columns = bardf.columns[2:len(bardf.columns)-1]
    
    if extra_var_bool[0]:
        bardf2['RealTime'] = bardf['RealTime'].astype('category')
        num_cell_day = len(bardf2['RealTime'].tolist())
    if extra_var_bool[1]:
        bardf2['Barcode'] = bardf['Barcode'].astype('category')
    if extra_var_bool[2]:
        bardf2['Pseudotime'] = bardf['Pseudotime']
    #bardf2 = bardf2.dropna()
    COLS = bardf2.columns.tolist()[1:]
    #print(COLS)
    
    bardf2 = bardf2.dropna()
    bardf2.index = [i for i in range(bardf2.shape[0])]
    bardf_train = bardf2[:-(round(num_cell_day*frac_val))]
    bardf_val = bardf2[len(bardf2) - (round(num_cell_day*frac_val)):]
    #print(bardf_val.shape, bardf_train.shape)
    
    train_y = bardf_train['Survival']
    train_data = bardf_train.iloc[:,1:]
    val_y = bardf_val['Survival'].tolist()
    val_data = bardf_val.iloc[:,1:]
    
    if frac_val == 0:
        train_data = bardf2.iloc[:,1:]
        train_y = bardf2['Survival']
    
    # Training data
    X_train, y_train = [], []
    for i in range(seq_len, len(train_data)):
      X_train.append(train_data[i-seq_len:i]) 
      y_train.append(train_y[i]) #Value of 0th column (Survival) of df-row seq_len+1
    X_train, y_train = np.array(X_train), np.array(y_train)
    
    if frac_val != 0:
    
        # Validation data
        X_val, y_val = [], []
        for i in range(seq_len, len(val_data)):
          X_val.append(val_data[i-seq_len:i]) 
          y_val.append(val_y[i]) #Value of 0th column (Survival) of df-row seq_len+1
        X_val, y_val = np.array(X_val), np.array(y_val)
    
        print('Transformer Training/Val set shapes (varies depending on barcode) ', X_train.shape, X_val.shape)
        return X_train, y_train, X_val, y_val, COLS
    
    else:
        
        print('Transformer Training/Val set shapes (varies depending on barcode) ', X_train.shape)
        return X_train, y_train, COLS

In [7]:
'''Create training and test data'''

def regr_train_test_split_dayiter(bar, frac_val, extra_var_bool, splitval):
    
    bardf = pd.DataFrame(data[bar])
    #bardf = bardf[bardf['RealTime'] <= exp_days[d]]
    num_cell_day = len(bardf['RealTime'].tolist())
    snum = int(num_cell_day*splitval)
    bardf = bardf.iloc[:snum,:]
    
    bardf = bardf.iloc[:,:len(bardf.columns)-1]
    bardf['RealTime'] = bardf['RealTime'].astype('category')
    bardf['Barcode'] = [bar_dict[bar] for _ in range(bardf.shape[0])]
    bardf['Barcode'] = bardf['Barcode'].astype('category')
    
    # normalize after train/test split
    scaler = MinMaxScaler(feature_range=(-1, 1)) 
    bardf2 = pd.DataFrame(scaler.fit_transform(bardf.iloc[:,2:len(bardf.columns)-1])) # survival and expression
    bardf2.columns = bardf.columns[2:len(bardf.columns)-1]
    
    if extra_var_bool[0]:
        bardf2['RealTime'] = bardf['RealTime'].astype('category')
        num_cell_day = len(bardf2['RealTime'].tolist())
    if extra_var_bool[1]:
        bardf2['Barcode'] = bardf['Barcode'].astype('category')
    if extra_var_bool[2]:
        bardf2['Pseudotime'] = bardf['Pseudotime']
        
    bardf2 = bardf2.dropna()
    bardf2.index = [i for i in range(bardf2.shape[0])]
    bardf_train = bardf2[:-(round(num_cell_day*frac_val))]
    bardf_val = bardf2[len(bardf2) - (round(num_cell_day*frac_val)):]
    #print(bardf_val)
    
    train_y = bardf_train['Survival']
    train_data = bardf_train.iloc[:,1:]
    val_y = bardf_val['Survival'].tolist()
    val_data = bardf_val.iloc[:,1:]
    
    # Training data
    X_train, y_train = np.array(train_data), np.array(train_y)
    
    # Validation data
    X_val, y_val = np.array(val_data), np.array(val_y)
    
    print('Regr Training/Val set shapes (varies depending on barcode) ', X_train.shape, X_val.shape)
    return X_train, y_train, X_val, y_val

In [8]:
########### testing function #################
# make the training and validation data

b = 'GFPBC_libB_92196'
frac_val = 0.20
extra_var_bool = [False, False, False] # 0-realtime, 1-barcode, 2-pseudotime
vs = [0.25, 0.5, 0.75, 1]
pct_test = frac_val/len(vs)
for v in vs:
    X_train, y_train, X_val, y_val, cols = transformer_train_test_split_dayiter(b, pct_test, extra_var_bool, v)
    X_train, y_train, X_val, y_val = regr_train_test_split_dayiter(b, pct_test, extra_var_bool, v)

X_full, y_full, COLS = transformer_train_test_split_dayiter(b, 0, extra_var_bool, 1)

Transformer Training/Val set shapes (varies depending on barcode)  (690, 8, 72) (167, 8, 72)
Regr Training/Val set shapes (varies depending on barcode)  (698, 72) (175, 72)
Transformer Training/Val set shapes (varies depending on barcode)  (1564, 8, 72) (167, 8, 72)
Regr Training/Val set shapes (varies depending on barcode)  (1572, 72) (175, 72)
Transformer Training/Val set shapes (varies depending on barcode)  (2438, 8, 72) (167, 8, 72)
Regr Training/Val set shapes (varies depending on barcode)  (2446, 72) (175, 72)
Transformer Training/Val set shapes (varies depending on barcode)  (3312, 8, 72) (167, 8, 72)
Regr Training/Val set shapes (varies depending on barcode)  (3320, 72) (175, 72)
Transformer Training/Val set shapes (varies depending on barcode)  (3487, 8, 72)


In [9]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, num_attention_blocks, num_transformer_blocks, dropout=0):
    if num_transformer_blocks == 0 and num_attention_blocks == 0:
        return inputs
    
    if num_attention_blocks != 0:
        for _ in range(num_attention_blocks):
            # Normalization and Attention
            # "EMBEDDING LAYER"
            x = layers.LayerNormalization(epsilon=1e-6)(inputs)

            # "ATTENTION LAYER"
            x = layers.MultiHeadAttention(
                key_dim=head_size, num_heads=num_heads, dropout=dropout
            )(x, x)
            x = layers.Dropout(dropout)(x)
            res = x + inputs
            
    if num_attention_blocks == 0:
        res = inputs
    
    if num_transformer_blocks != 0: 
        for _ in range(num_transformer_blocks):
            # FEED FORWARD Part - you can stick anything here or just delete the whole section - it will still work. 
            x = layers.LayerNormalization(epsilon=1e-6)(res)
            x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation = "relu")(x)
            x = layers.Dropout(dropout)(x)
            x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    
    # may need to return something else if transformer blocks are 0
    return x + res

In [10]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_attention_blocks,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0, 
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    
    # This is what stacks our transformer blocks
    x = transformer_encoder(x, head_size, num_heads, ff_dim, num_attention_blocks, num_transformer_blocks, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="elu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1, activation="linear")(x) #this is a pass-through
    return keras.Model(inputs, outputs)

In [11]:
def lr_scheduler(epoch, lr, warmup_epochs=30, decay_epochs=100, initial_lr=1e-6, base_lr=1e-3, min_lr=5e-5):
    if epoch <= warmup_epochs:
        pct = epoch / warmup_epochs
        return ((base_lr - initial_lr) * pct) + initial_lr

    if epoch > warmup_epochs and epoch < warmup_epochs+decay_epochs:
        pct = 1 - ((epoch - warmup_epochs) / decay_epochs)
        return ((base_lr - min_lr) * pct) + min_lr

    return min_lr

# This learning rate scheduler is from Mr. Theodoros Ntakouris' article at https://towardsdatascience.com/the-time-series-transformer-2a521a0efad3

In [12]:
callbacks = [
            keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
            keras.callbacks.LearningRateScheduler(lr_scheduler)
            ]

In [13]:
from keras.utils.vis_utils import plot_model
import pydot
import graphviz

#tf.keras.utils.plot_model(model, to_file ='model_plot.png', show_shapes = True, show_layer_names = True)

In [14]:
def plot_transformer_predictions_losses(X_train, y_train, X_val, y_val, bar, dir_path, model):
    
    #Calculate predication for training, validation and test data
    print(X_train.shape, X_val.shape)
    train_pred = model.predict(X_train)
    val_pred = model.predict(X_val)

    #Print evaluation metrics for all datasets
    train_mse = mean_squared_error(y_train, train_pred)
    val_mse = mean_squared_error(y_val, val_pred)
    print(' ')
    print('Transformer Evaluation metrics')
    print('Transformer Training MSE - MSE: {:.4f}'.format(train_mse)) ### these lines should be MSE???
    print('Transformer Validation MSE - MSE: {:.4f}'.format(val_mse))
    

    plt.plot(y_train, label='Survival')
    plt.plot(np.arange(train_pred.shape[0]), train_pred, linewidth=3, label='Predicted Survival')
    plt.title("Transformer Training Data "+bar, fontsize=14)
    plt.xlabel('Real + Pseudotime')
    plt.ylabel('Survival')
    plt.ylim(-1.5, 1.5)
    plt.legend(loc="best", fontsize=12)
    plt.savefig(dir_path+bar+'_training.png', bbox_inches='tight')
    plt.show()

    #Plot validation data results
    plt.plot(y_val, label='Survival')
    plt.plot(np.arange(val_pred.shape[0]), val_pred, linewidth=3, label='Predicted Survival')
    plt.title("Transformer Validation Data "+bar, fontsize=14)
    plt.xlabel('Real + Pseudotime')
    plt.ylabel('Survival')
    plt.ylim(-1.5, 1.5)
    plt.legend(loc="best", fontsize=12)
    plt.savefig(dir_path+bar+'_validation.png', bbox_inches='tight')
    plt.show()
    
    return train_mse, val_mse

In [15]:
def plot_regr_predictions_losses(X, y, X_test, y_test, bar, dir_path, regr):
    
    #Calculate predication for training, validation and test data
    train_pred = regr.predict(X)
    val_pred = regr.predict(X_test)

    #Print evaluation metrics for all datasets
    train_eval = mean_squared_error(y, train_pred)
    val_eval = mean_squared_error(y_test, val_pred)
    print(' ')
    print('Evaluation metrics Regr')
    print('Regr Training Data - MSE: {:.4f}'.format(train_eval)) ### these lines should be MSE???
    print('Regr Validation Data - MSE: {:.4f}'.format(val_eval))
    
    plt.plot(y, label='Survival')
    plt.plot(np.arange(train_pred.shape[0]), train_pred, linewidth=3, label='Predicted Survival')
    plt.title("Regr Training Data "+bar, fontsize=14)
    plt.xlabel('Real + Pseudotime')
    plt.ylabel('Survival')
    plt.ylim(-1.5, 1.5)
    plt.legend(loc="best", fontsize=12)
    plt.savefig(dir_path+bar+'_training.png', bbox_inches='tight')
    plt.show()

    #Plot validation data results
    plt.plot(y_test, label='Survival')
    plt.plot(np.arange(val_pred.shape[0]), val_pred, linewidth=3, label='Predicted Survival')
    plt.title("Regr Validation Data "+bar, fontsize=14)
    plt.xlabel('Real + Pseudotime')
    plt.ylabel('Survival')
    plt.ylim(-1.5, 1.5)
    plt.legend(loc="best", fontsize=12)
    plt.savefig(dir_path+bar+'_validation.png', bbox_inches='tight')
    plt.show()
    
    return train_eval, val_eval

In [16]:
def SHAPvals4(X_val, y_val, bar, COLS, dir_path, model):
    
    #COLS = bardf.columns.tolist()[1:] # may need fixing
    #print(len(COLS), X_val.shape)
    #print(COLS)

    results = []
    print(' Computing feature importance...'+bar)

    # COMPUTE BASELINE (NO SHUFFLE)
    oof_preds = model.predict(X_val, verbose=0).squeeze() 
    baseline_mae = np.mean(np.abs( oof_preds-y_val ))
    results.append({'feature':'BASELINE','mae':baseline_mae, 'prediction':oof_preds.tolist(), 'true':y_val.tolist()})           

    for k in range(len(COLS)):

        if k in [20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]:
            print('SHAP', k, ' '+bar)
        # SHUFFLE FEATURE K
        save_col = X_val[:,:,k].copy()
        mae_all = []
        pred_df = pd.DataFrame() #########
        for i in range(30):
            np.random.shuffle(X_val[:,:,k])

            # COMPUTE OOF MAE WITH FEATURE K SHUFFLED
            oof_preds = model.predict(X_val, verbose=0).squeeze() 
            pred_df['prediction_'+str(i)] = oof_preds #######
            mae = np.mean(np.abs( oof_preds-y_val ))
            mae_all.append(mae)
            X_val[:,:,k] = save_col
        results.append({'feature':COLS[k],'mae_mean':np.mean(mae_all), 'mae_std':np.std(mae_all), 'mae_all':mae_all})
        for each_col in pred_df.columns.tolist():
            results[-1][each_col] = pred_df[each_col].tolist()

    # SAVE FEATURE IMPORTANCE
    df = pd.DataFrame(results)
    df = df.sort_values('mae_mean',ascending=False)
    df['fc_mae'] = df['mae'] - baseline_mae
    df.to_csv(dir_path+bar+'_feature_importance_full.csv',index=False)
    print('done with '+bar)

In [17]:
# set seeds produces more similar results, but not exact
seed_value = 42
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

num_layers = 10
n_heads = 40 # was 40  # Number of attention heads # no difference or imporvement between 5-60
n_head_size= 5 # was 46 # Embedding size for attention # no difference between 5-46
n_ff_dim = 5  # was 55    # Hidden layer size in feed forward network inside transformer # some difference between 5-55

batch_size = 32 # was 32
seq_len = 8 # was 8

In [18]:
# remember to change the output file names to the appropriate path

pfi = True # if you don't want to compute permutation feature importance (lengthy) change to false
frac_val = 0.20 # percent of data to be used for testing
#test_barcodes = ['GFPBC_libB_92196'] 
extra_var_bool = [False, False, False] # include 0-realtime, 1-barcode, 2-pseudotime as features for training
intervals = [0.25, 0.5, 0.75, 1]
pct_test = frac_val/len(intervals) # comment out and use frac_val for last 20%
#intervals = [1] # if you only want the final test section as validation
#pct_test = frac_val/4 # if you only want the final test section as validation

mse_dict = {} # structure is {barcode: {day_0: [trans_train, trans_val, regr_train, regr_val], day_n...}, barcode_n: ...}

    
def runDynamicDeepLearn(bar):
    
    print('\n\n')
    print(bar)
    mse_dict[bar] = {}
    
    for v in intervals:
        # make the training and validation data
        X_train, y_train, X_val, y_val, COLS = transformer_train_test_split_dayiter(bar, pct_test, extra_var_bool, v)

        # build and compile full model
        ### the main chunk ###
        input_shape = X_train.shape[1:]
        model = build_model(
            input_shape,
            head_size=n_head_size, # Embedding size for attention
            num_heads=n_heads, # Number of attention heads
            ff_dim=n_ff_dim, # Hidden layer size in feed forward network inside transformer
            num_attention_blocks=num_layers, # 3 iterations for attention
            num_transformer_blocks=num_layers, # six layers doesn't add or subtract much so best to keep simple
            mlp_units=[256],
            mlp_dropout=0.4, # unclear if increasing or decreasing has sizable effect
            dropout=0.14 # unclear if increasing or decreasing has sizable effect
        )
        model.compile(
            loss="mean_squared_error",
            optimizer=keras.optimizers.Adam(learning_rate=1e-3),  # was 1e-4
            metrics=["mean_squared_error"],
        )
        # fit the model
        history = model.fit(X_train, y_train, 
                        batch_size=batch_size, 
                        epochs=20,
                        validation_data=(X_val, y_val)) 
        
        # build no attention model
        noattn_model = build_model(
            input_shape,
            head_size=n_head_size, # Embedding size for attention
            num_heads=n_heads, # Number of attention heads
            ff_dim=n_ff_dim, # Hidden layer size in feed forward network inside transformer
            num_attention_blocks=0, # number of iteration layers for attention
            num_transformer_blocks=num_layers, # six layers doesn't add or subtract much so best to keep simple
            mlp_units=[256],
            mlp_dropout=0.4, # unclear if increasing or decreasing has sizable effect
            dropout=0.14 # unclear if increasing or decreasing has sizable effect
        )
        noattn_model.compile(
            loss="mean_squared_error",
            optimizer=keras.optimizers.Adam(learning_rate=1e-3),  # was 1e-4
            metrics=["mean_squared_error"],
        )
        # fit the model
        noattn_history = noattn_model.fit(X_train, y_train, 
                        batch_size=batch_size, 
                        epochs=20,
                        validation_data=(X_val, y_val))
        
        # build no encode model
        noenc_model = build_model(
            input_shape,
            head_size=n_head_size, # Embedding size for attention
            num_heads=n_heads, # Number of attention heads
            ff_dim=n_ff_dim, # Hidden layer size in feed forward network inside transformer
            num_attention_blocks=num_layers, # number of iteration layers for attention
            num_transformer_blocks=0, # six layers doesn't add or subtract much so best to keep simple
            mlp_units=[256],
            mlp_dropout=0.4, # unclear if increasing or decreasing has sizable effect
            dropout=0.14 # unclear if increasing or decreasing has sizable effect
        )
        noenc_model.compile(
            loss="mean_squared_error",
            optimizer=keras.optimizers.Adam(learning_rate=1e-3),  # was 1e-4
            metrics=["mean_squared_error"],
        )
        # fit the model
        noenc_history = noenc_model.fit(X_train, y_train, 
                        batch_size=batch_size, 
                        epochs=20,
                        validation_data=(X_val, y_val))


        if X_val.shape[0] > 0:

            # predictions and loss
            train_mse, val_mse = plot_transformer_predictions_losses(X_train, y_train, X_val, y_val, bar, 'test/transformer_ptime_', model)

            # noattn - predictions and loss
            noattn_train_mse, noattn_val_mse = plot_transformer_predictions_losses(X_train, y_train, X_val, y_val, bar, 'test/noattn_ptime_', noattn_model)
            
            # noenc - predictions and loss
            noenc_train_mse, noenc_val_mse = plot_transformer_predictions_losses(X_train, y_train, X_val, y_val, bar, 'test/noenc_ptime_', noenc_model)
            
            # build multivariate regression model
            X, y, X_test, y_test = regr_train_test_split_dayiter(bar, pct_test, extra_var_bool, v)
            regr = linear_model.LinearRegression()
            regr.fit(X, y)
            regr_train_mse, regr_val_mse = plot_regr_predictions_losses(X, y, X_test, y_test, bar, 'test/regr_ptime_', regr)

            # add to mse dictionary
            mse_dict[bar][v] = [train_mse, val_mse, regr_train_mse, regr_val_mse, noattn_train_mse, noattn_val_mse, noenc_train_mse, noenc_val_mse]

            # SHAP values
            if v == 1 and pfi == True:
                X_full, y_full, COLS = transformer_train_test_split_dayiter(bar, 0, extra_var_bool, 1)
                SHAPvals4(X_full, y_full, bar, COLS, 'test/transformer_full', model)
            sys.stdout.flush()
                

    return mse_dict

In [ ]:
import multiprocessing
import sys
from multiprocessing import Pool
threads = 14
print('running these barcodes: ', new_barcodes)

    
pool = Pool(threads)
result = pool.starmap(runDynamicDeepLearn, zip(new_barcodes)) 
print(result)


print(mse_dict)

running these barcodes:  ['GFPBC_libB_92196', 'GFPBC_libB_20182', 'GFPBC_libB_90850', 'GFPBC_libB_10678', 'GFPBC_libB_37798', 'GFPBC_libB_31581', 'GFPBC_libB_40758', 'GFPBC_libB_94397', 'GFPBC_libB_83026', 'GFPBC_libB_10737']
























GFPBC_libB_37798



GFPBC_libB_20182
GFPBC_libB_90850GFPBC_libB_10678GFPBC_libB_92196GFPBC_libB_83026GFPBC_libB_40758


GFPBC_libB_10737
GFPBC_libB_94397





GFPBC_libB_31581
Transformer Training/Val set shapes (varies depending on barcode) Transformer Training/Val set shapes (varies depending on barcode)   (53, 8, 72)Transformer Training/Val set shapes (varies depending on barcode) (92, 8, 72)  (92, 8, 72)(17, 8, 72) 
(17, 8, 72)
Transformer Training/Val set shapes (varies depending on barcode)   (130, 8, 72)(8, 8, 72) Transformer Training/Val set shapes (varies depending on barcode) 
 (27, 8, 72)(229, 8, 72)
 Transformer Training/Val set shapes (varies depending on barcode) Transformer Training/Val set shapes (varies depending on barcode

2024-01-16 11:21:53.838939: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-01-16 11:21:53.908965: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-01-16 11:21:53.908965: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-01-16 11:21:53.917133: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-01-16 11:21:53.958998: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-01-16 11:21:53.959006: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ER

In [ ]:
mse_dict = {}
for i, each in enumerate(result):
    mse_dict[new_barcodes[i]] = each[new_barcodes[i]]

mse = pd.DataFrame(mse_dict)
mse

In [ ]:
# write the mse data
f = open('mse/test_mse.csv', 'w')
f.write(str(mse_dict))
f.close()

In [ ]:
# take a look at it
mse_file = open('mse/test_mse.csv', 'r')
for line in mse_file:
    mse = eval(line)
mse